# Корреляционный анализ

## Вопрос 1

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

Ответ: -0.6548

In [2]:
import pandas as pd
import numpy as np

In [6]:
water = pd.read_csv('water.txt', delimiter='\t')
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [9]:
print(np.round(water.corr()['mortality']['hardness'], 4))

-0.6548


## Вопрос 2

В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.

Ответ: -0.6317

In [10]:
print(np.round(water.corr('spearman')['mortality']['hardness'], 4))

-0.6317


## Вопрос 3

Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

Ответ: -0.3686

In [21]:
corr_south_pirson = np.round(water[water['location'] == 'South'].corr()['mortality']['hardness'], 4)
corr_north_pirson = np.round(water[water['location'] == 'North'].corr()['mortality']['hardness'], 4)
corr_pirson = [corr_north_pirson, corr_south_pirson]
print(corr_pirson)
print(corr_pirson[np.argmin(np.abs(corr_pirson))])

[-0.36859999999999998, -0.60219999999999996]
-0.3686


## Вопрос 4

Среди респондентов `General Social Survey` 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

$MCC_{X_1X_2} = \frac{ad-bc}{\sqrt{(a+b)(a+c)(b+d)(c+d)}}$

ответ: -0.109

In [28]:
a = 203
b = 718
c = 239
d = 515

def mcc(a,b,c,d):
    return (a*d - b*c) / np.sqrt((a+b)*(a+c)*(b+d)*(c+d))
    
print(np.round(mcc(a,b,c,d), 3))

-0.109


## Вопрос 5

В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию `scipy.stats.chi2_contingency`. Введите номер первой значащей цифры (например, если вы получили $5.5*10^{−8}$, нужно ввести 8).

Ответ: 5

In [34]:
from scipy.stats import chi2_contingency

chi2, p, dof, ex = chi2_contingency([[203, 718],[239,515]])
p

1.0558987006638725e-05

## Вопрос 6

В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

Ответ: 0.0539

In [55]:
from scipy.stats import norm

p1 = 239 / (239 + 515)
n1 = 239 + 515
p2 = 203 / (203 + 718)
n2 = 203 + 718

z = norm.ppf(1 - 0.05 / 2)
left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)

np.round([left_boundary, right_boundary], 4)

array([ 0.0539,  0.1392])

## Вопрос 7

Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

Ответ: 6

In [58]:
def proportions_diff_z_stat_ind(p1,p2,n1,n2):
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - norm.cdf(z_stat)
    
proportions_diff_z_test(proportions_diff_z_stat_ind(p1,p2,n1,n2))

8.1534530895766011e-06

## Вопрос 8

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

||Не доволен|Более или менее|Доволен|
|---|---|---|---|
|Не очень счастлив|197|111|33|
|Достаточно счастлив|382|685|331|
|Очень счастлив|110|342|333|

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

Ответ: 293.6831

In [42]:
conf_matrix = np.array([[197,111,33],[382,685,331],[110,342,333]])
chi2, p, dof, ex = chi2_contingency(conf_matrix)
print(chi2, p, dof, ex)
print(np.round(chi2, 4))

293.683110397 2.49642995801e-62 4 [[  93.08597464  153.74722662   94.16679873]
 [ 381.6251981   630.318542    386.0562599 ]
 [ 214.28882726  353.93423138  216.77694136]]
293.6831


## Вопрос 9

На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

Ответ: 62

## Вопрос 10

Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

$\phi_c(X_1^n, X_2^n) = \sqrt{\frac{\chi^2(X_1^n, X_2^n)}{n(min(K_1,K_2)-1)}}$

In [47]:
n=conf_matrix.sum()
coef = np.sqrt(chi2 / (n*(min(conf_matrix.shape)-1)))
print(np.round(coef,4))

0.2412
